In [ ]:
# Import important libraries 
import keras
import seaborn as sns
from keras import layers
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, AveragePooling2D,SeparableConv2D,GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pandas as pd
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Activation,BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import LeakyReLU
from scipy.stats import boxcox
from PIL import Image
from tensorflow.keras import backend as K
import re
import os
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()
scaler5 = MinMaxScaler()
scaler6 = MinMaxScaler()
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import model_from_json
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:

df = pd.read_csv('.......') #Read Datafrom file 

df1=df[['PRODX','PRODY','INJX','INJY']] #extract well coordinates only (x-train)


In [ ]:
data = df['COP']
plt.hist(data, bins=20, color='blue', alpha=0.7)


# Customize the plot
plt.title('Histogram of Data')
plt.xlabel('Values')
plt.ylabel('Frequency')

# Show the data distribution in a histogram
plt.show()

In [ ]:
# Load Images of realization
for i in range(len(df1)):
    image2 = Image.open(df['R'][i])

    image2.save(f".......")
image_data_directory = '.......'

# Get a list of all image files in the directory
all_image_files = [os.path.join(image_data_directory, filename) for filename in os.listdir(image_data_directory)]
all_image_files.sort(key=lambda x: int(re.search(r'\d+', os.path.basename(x)).group()))
pics = all_image_files


train_pics = []
#Read images and normalize the values
for i in range(len(pics)):
    img = cv2.imread(pics[i] )  #,cv2.IMREAD_GRAYSCALE
    img=cv2.resize(img, (60,60))/255
    train_pics.append(img)
    
train_picss= np.array(train_pics)
train_picss.shape

In [ ]:
#Aplit the data into 60% Training, 20% Validation, 20% Testing
split = train_test_split(df['COP'], train_picss,df1, test_size=0.4, random_state=42)
(y_train,y_test1, pic_train, pic_test1,cord_train,cord_test1) = split
split1 = train_test_split(y_test1, pic_test1,cord_test1, test_size=0.5, random_state=42)
(y_test,y_val, pic_test, pic_val,cord_test,cord_val) = split1
len(pic_train), len(y_train), len(pic_test),len(pic_test), len(y_test), len(cord_train), len(cord_test),len(y_val)

split3 = {'Pic Train':len(pic_train), 'Pic Validation': len(pic_val),'Pic Test':len(pic_test),
         'cord Train':len(pic_train), 'cord Validation': len(cord_val),'cord Test':len(cord_test),
         'y Train':len(y_train), 'y Validation': len(y_val),'y Test':len(y_test)
        }


my_dict_of_lists = {'data': ['Pic', 'cord', 'COP '],
                    'Train': [len(pic_train), len(cord_train), len(y_train)],
                    'Val':  [len(pic_val), len(cord_val), len(y_val)],
                    'Test':  [len(pic_test), len(pic_test), len(y_test)]
                   }

# Convert the dictionary of lists to a DataFrame
d = pd.DataFrame(my_dict_of_lists)

# Display the DataFrame
print(d)

In [ ]:
#normalizing all well locations 
cord_train = scaler5.fit_transform(cord_train)
cord_test = scaler6.fit_transform(cord_test)
cord_val = scaler1.fit_transform(cord_val)
y_testt = y_test.values
y_testtt = y_testt.reshape(-1,1)
testY= scaler2.fit_transform(y_testtt)
y_trainn = y_train.values
y_trainnn = y_trainn.reshape(-1,1)
trainY= scaler3.fit_transform(y_trainnn)
y_vall = y_val.values
y_valll = y_vall.reshape(-1,1)
valY=  scaler4.fit_transform(y_valll)

In [ ]:
# check each set statistics 
print("Training set statistics:")
print("Min:", np.min(trainY))
print("Max:", np.max(trainY))
print("Quartiles:", np.percentile(trainY, [25, 50, 75]))

print("\nValidation set statistics:")
print("Min:", np.min(valY))
print("Max:", np.max(valY))
print("Quartiles:", np.percentile(valY, [25, 50, 75]))

print("\nTest set statistics:")
print("Min:", np.min(testY))
print("Max:", np.max(testY))
print("Quartiles:", np.percentile(testY, [25, 50, 75]))

In [ ]:
# To create initial NN model 

def build_multi_input_model(hp):
    # CNN branch
    cnn_input = Input(shape=(60, 60, 3))  # Adjust to your image input shape
    x = cnn_input
    for i in range(1, hp.Int('num_cnn_layers', 1, 4) + 1):  # Loop to add variable number of conv layers
        x = Conv2D(filters=hp.Int(f'conv_{i}_filters', 16, 128, step=16),
                   kernel_size=hp.Choice(f'conv_{i}_kernel_size', [3, 5]),
                   activation=hp.Choice("activation", ["linear","relu", "tanh"]), padding='same')(x)
        x = BatchNormalization()(x)
        pool_size = hp.Choice(f'pool_{i}_size', [2, 3,4])
        x = MaxPooling2D(pool_size=(pool_size, pool_size))(x)
        
    cnn_output = Flatten()(x)

    # ANN branch
    cnn2_input = Input(shape=(60,60,1))  # Adjust to your tabular input shape (number of features)
    y = cnn2_input
    for i in range(1, hp.Int('num_cnn2_layers', 1, 4) + 1):  # Loop to add variable number of conv layers
        y = Conv2D(filters=hp.Int(f'conv2_{i}_filters', 16, 128, step=16),
                   kernel_size=hp.Choice(f'conv2_{i}_kernel_size', [3, 5]),
                   activation='relu', padding='same')(y)
        y = BatchNormalization()(y)
        pool_size = hp.Choice(f'pool_{i}_size', [2, 3,4])
        y = MaxPooling2D(pool_size=(pool_size, pool_size))(y)
        
    cnn2_output = Flatten()(y)
    for i in range(1, hp.Int('num_ann_layers', 1, 8) + 1):  # Loop to add variable number of dense layers
        y = Dense(units=hp.Int(f'cnn2_{i}_units', 16, 512, step=16), activation='relu')(cnn2_output)
        if hp.Boolean(f'ann_{i}_dropout'):  # Optionally add a dropout layer
            y = Dropout(rate=hp.Float(f'cnn2_{i}_dropout_rate', 0.1, 0.5, step=0.1))(y)
    

    # Combine branches
    combined = concatenate([cnn_output, cnn2_output])

    # Fully connected layers after merging
    z = combined
    for i in range(1, hp.Int('num_fc_layers', 1, 8) + 1):
        z = Dense(units=hp.Int(f'fc_{i}_units', 16, 512, step=16), activation=hp.Choice("activation", ["linear","relu", "tanh"]))(z)
        if hp.Boolean(f'fc_{i}_dropout'):  # Optionally add a dropout layer
            z = Dropout(rate=hp.Float(f'fc_{i}_dropout_rate', 0.1, 0.5, step=0.1))(z)

    # Output layer
    output = Dense(1, activation=hp.Choice("activation", ["linear","relu", "tanh"]))(z)  # Adjust 'num_classes' based on your task

    model = Model(inputs=[cnn_input, cnn2_input], outputs=output)

    # Compile model
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

    # Optionally, you can also tune the learning rate of each optimizer

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

    # Compile the model with the chosen optimizer, loss, and metrics
    model.compile(optimizer=optimizer, loss='msle', metrics=[R2])


    return model

import keras_tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=build_multi_input_model,
    # Specify the name and direction of the objective.
    objective=keras_tuner.Objective("val_R2", direction="max"),
    max_trials=100,
    overwrite=True,
    directory="my_dir",
    project_name="custom_metrics",
)

In [ ]:
#optimize NN model to get highest R^2
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor the validation loss
                               patience=15, # Number of epochs with no improvement after which training will be stopped
                               restore_best_weights=True)
tuner.search([pic_train,cord_train1], trainY, epochs=200, validation_data=([pic_val,cord_val1], valY), callbacks=[early_stopping])

In [ ]:
#to get the best model from keras tuner 
tuner_model = tuner.get_best_models(num_models=1)[0]
hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
for param in best_hyperparameters.values:
    print(f"{param}: {best_hyperparameters.get(param)}")

In [ ]:
#Coeff. Determination 
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

In [ ]:
#Use for retraining if needed
pic_all = np.concatenate((pic_train, pic_val))
cord_all = np.concatenate((cord_train, cord_val))
allY = np.concatenate((trainY, valY))

In [ ]:
#NN model retraining if needed
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor the validation loss
                               patience=15, # Number of epochs with no improvement after which training will be stopped
                               restore_best_weights=True)
opt = Adam(learning_rate=0.00010904442257839217)
tuner_model.compile(loss='msle', optimizer=opt, metrics = [R2,'mae'] )
print("[INFO] training model...")
history = tuner_model.fit(x=[pic_train,cord_train], y=trainY, 
     validation_data = ([pic_val,cord_val], valY),
        epochs=250, batch_size = 64, callbacks = [early_stopping], verbose = 2)

In [ ]:
#check overfitting
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
#R^2 testing
preds = tuner_model.predict([pic_test,cord_test])
data = testY
r2 = r2_score(preds, data)

print(f'R2 Score : {round(r2,2)}')
# Plot true values vs. predicted values

plt.scatter(data, preds, label=f'R² = {r2:.2f}')
plt.plot([0, 1], [0, 1], '--', color='red', label='Ideal Line')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs. Predicted Values with R² Score')
plt.legend()
plt.show()

In [ ]:
#R^2 training
preds = tuner_model.predict([pic_train,cord_train])
data = trainY
r2 = r2_score(preds, data)

print(f'R2 Score : {round(r2,2)}')
# Plot true values vs. predicted values

plt.scatter(data, preds, label=f'R² = {r2:.2f}')
plt.plot([0, 1], [0, 1], '--', color='red', label='Ideal Line')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs. Predicted Values with R² Score')
plt.legend()
plt.show()

In [ ]:
#preprocess PSO
olumns = ['PRODX', 'PRODY', 'INJX','INJY']
df2 = pd.DataFrame(scaler5.inverse_transform(np.array(cord_train)), columns = columns)
df2
maxp = df2['PRODX'].max()
minp = df2['PRODX'].min()

maxpy = df2['PRODY'].max()
minpy = df2['PRODY'].min()

maxi = df2['INJX'].max()
mini = df2['INJX'].min()

maxiy = df2['INJY'].max()
miniy = df2['INJY'].min()

In [ ]:
#Generate inside reservoir only
image = cv2.imread('.......', cv2.IMREAD_GRAYSCALE)
image=cv2.resize(image, (60,60))
# Threshold the image to get a binary mask
# (this threshold value is just an example, you might need to adjust it)
_, mask = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

# Get the dimensions of the image to limit the random point generation
height, width = mask.shape

# Function to generate a point inside the leaf
def generate_and_normalize_well_locations(num_wells):
    """
    Generates multiple sets of well location coordinates that fall within the leaf's area in the mask.
    Normalizes these coordinates based on provided min and max values for each axis.
    """
    height, width = mask.shape
    all_well_locations = []

    for _ in range(num_wells):
        well_locations = []
        for _ in range(1):  # Generate two wells (four points)
            while True:
                x1 = np.random.randint(2, width)
                y1 = np.random.randint(2, height)
                if mask[y1, x1]:  # Assuming leaf area is True in the mask
                    # Normalize the coordinates
                    px = (x1 - minp) / (maxp - minp)
                    py = (y1 - minpy) / (maxpy - minpy)
                    well_locations.extend([px, py])
                    break
        
        # Assuming you want to normalize second set of points differently
        # using mini, maxi, miniy, maxiy
        for _ in range(1):  # Generate two more wells (four more points)
            while True:
                x = np.random.randint(2, width)
                y = np.random.randint(2, height)
                if mask[y, x]:  # Check if within leaf
                    # Normalize the coordinates
                    ix = (x - mini) / (maxi - mini)
                    iy = (y - miniy) / (maxiy - miniy)
                    well_locations.extend([ix, iy])
                    break

        all_well_locations.append(well_locations)

    return np.array(all_well_locations)

In [ ]:
#Check if inside reservoir only
def check_coordinates_in_leaf(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    height, width = mask.shape
    
    # Check if both coordinates are within the image dimensions
    x1, y1= int(round(x1)), int(round(y1))
    x2, y2= int(round(x2)), int(round(y2))
    if 0 <= x1 < width and 0 <= y1 < height and 0 <= x2 < width and 0 <= y2 < height:
        # Check if both coordinates fall within the leaf area
        if mask[y1, x1] and mask[y2, x2]:
            return coord1, coord2
    return 0

In [ ]:
#Pics of relizations
R76 = train_picss[829:830]
R60 = train_picss[1400:1401]
R82 = train_picss[4:5]

In [ ]:
#Objective function for PSO
def objective_function(Xp,Yp,Xi,Yi):
    point1=  Xp,Yp
    point2 = Xi,Yi 
    check_coordinates_in_leaf(point1,point2) #Previously defined function to check if wells are inside reservoir
    if check_coordinates_in_leaf(point1, point2) == 0:
        return 0  # Points are outside the leaf
    Xp = (Xp - minp) / (maxp - minp) #Normalizing
    Yp = (Yp - minpy) / (maxpy - minpy)
    Xi = (Xi - mini) / (maxi - mini)
    Yi = (Yi - miniy) / (maxiy - miniy)
    well_locations = np.array((Xp,Yp,Xi,Yi))
    well_locations = well_locations.reshape(1, 4)
    well_locations1 = scaler5.inverse_transform(well_locations.reshape(1,-1))
    r82 = tuner_model.predict([R82,well_locations])
    r76 = tuner_model.predict([R76,well_locations])
    r60 = tuner_model.predict([R60,well_locations])
    productivity = (r82 + r76 + r60)/3
    return productivity.item()

In [ ]:
%%time

# Problem bounds
Xp_bounds = (0, 60)
Yp_bounds = (0, 60)
Xi_bounds = (0, 60)
Yi_bounds = (0, 60)

# PSO parameters
num_particles = 50
num_dimensions = 4  # (Xp, Yp), (Xi, Yi)
num_iterations = 200
w_max = 0.9  # Initial inertia weight
w_min = 0.4  # Final inertia weight
c1 = 1.79445  # Cognitive (particle) weight
c2 = 1.79445  # Social (swarm) weight

# Initialize well locations randomly within the bounds of the egg reservoir model
particles = np.random.rand(num_particles, num_dimensions)
particles *= np.array([Xp_bounds[1], Yp_bounds[1], Xi_bounds[1], Yi_bounds[1]])  # Scale particles to bounds
particles = particles.astype(float)  # Ensure particles are floating-point

# Initialize velocities as floating-point zeros
velocities = np.zeros((num_particles, num_dimensions), dtype=float)

# Initialize personal best well coordinates and COP
pbest = np.copy(particles)
pbest_scores = np.array([objective_function(*p) for p in particles])

# Initialize global best well locations(particles) and cop
gbest_index = np.argmax(pbest_scores)
gbest = particles[gbest_index]
gbest_score = pbest_scores[gbest_index]
best_obj_values = [] 
# PSO loop
for i in range(num_iterations):
    # Dynamic inertia weight adjustment
    w = w_max - ((w_max - w_min) * i / num_iterations)
    
    for j in range(num_particles):
        # Update velocities
        r1, r2 = np.random.rand(2)  # Random coefficients for stochastic components
        velocities[j] = w * velocities[j] + \
                        c1 * r1 * (pbest[j] - particles[j]) + \
                        c2 * r2 * (gbest - particles[j])

        # Update particle positions
        particles[j] += velocities[j]
        velocities[j] =velocities[j]
        # Ensure particles remain within bounds (optional, depending on your problem)
        # particles[j] = np.clip(particles[j], [Xp_bounds[0], Yp_bounds[0], Xi_bounds[0], Yi_bounds[0]], 
        #                                       [Xp_bounds[1], Yp_bounds[1], Xi_bounds[1], Yi_bounds[1]])

        # Evaluate new positions
        score = objective_function(*particles[j])

        # Update personal best if the new score is better
        if score > pbest_scores[j]:
            pbest[j] = particles[j]
            pbest_scores[j] = score

            # Update global best if the new score is better
            if score > gbest_score:
                gbest = particles[j]
                gbest_score = score
    current_best_value = scaler3.inverse_transform(np.array(gbest_score).reshape(-1,1))
    
    # Store the best objective value of the current iteration
    best_obj_values.append(current_best_value)
    #Print the best score and position found in this iteration
    print(f"Iteration {i + 1}: Best score = {gbest_score}, Best position = {gbest}")

# Final best solution
print(f"Final best position: {gbest}, Best score: {gbest_score}")




In [ ]:
scaler3.inverse_transform(np.array(gbest_score).reshape(-1,1))